# core

> Wraps global environment maintenance.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from functools import partial
import os
import sys
from pathlib import Path
import subprocess as sproc

import subprocess
from dataclasses import dataclass, field


@dataclass
class ProcResult:
    ok: bool
    out: str
    err: str
    raw: sproc.CompletedProcess = field(repr=False)


def sprun(*args, **kwargs) -> ProcResult:
    result = subprocess.run(
        map(str, args),
        capture_output=True,
        text=False,  # Keep raw output as bytes
        **kwargs
    )
    return ProcResult(
        ok=result.returncode == 0,
        out=result.stdout.decode('utf-8', errors='replace').strip() if result.stdout else "",
        err=result.stderr.decode('utf-8', errors='replace').strip() if result.stderr else "",
        raw=result,
    )


class GlobalEnv:
    """instance of a globally managed environment
    """

    BASE = Path("~/tk/uv").expanduser() 

    def __init__(self, name: str):
        self.name = name
        self.path = self.BASE / name

    @property
    def activate(self) -> Path:
        shell = os.path.basename(os.getenv('SHELL', '/bin/bash'))
        for n, activate_file in ({
            'bash': 'activate',
            'zsh': 'activate',
            'fish': 'activate.fish',
            'csh': 'activate.csh',
        }).items():
            if shell.startswith(n):
                break  # fix for "fishlogin"
        return self.path / "bin" / activate_file
    
    def exists(self):
        return self.path.exists()

    def feats(env):
        return dict(
            dir_exists = env.path.exists(),
            has_activate = env.activate.exists(),
            #has_py = (env.path / "bin" / "python").exists(),
            py = sprun(
                str(env.path / "bin" / "python"),
                "--version"
            ).out,
        )


info = partial(print, file=sys.stderr)
tell = lambda *a: print(*a) if any(a) else None


def make_or_activate(name: str):
    outs = None
    env = GlobalEnv(name)

    if env.exists():
        info(f"Found {env.feats()=}")
    else:
        created = sprun("uv", "venv", env.path)
        info(f"{created.ok=}")

    if all((feats := env.feats()).values()):
        outs = f"{env.activate}"
    else:
        info(f"Feat error: {feats}")

    return env, outs


def cli():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("name", help="Environment name")
    args = parser.parse_args()
    env, outs = make_or_activate(args.name)
    tell(outs)
    return env

# cli()
# CompletedProcess(args=['uv', 'venv', '/Users/toni/tk/uv/pygen'], returncode=0, stdout=b'', stderr=b'Using Python 3.11.6 interpreter at: \x1b[36m/Users/toni/miniconda3/envs/research/bin/python3.11\x1b[39m\nCreating virtualenv at: \x1b[36m/Users/toni/tk/uv/pygen\x1b[39m\n')


usage: ipykernel_launcher.py [-h] name
ipykernel_launcher.py: error: the following arguments are required: name


SystemExit: 2

/Users/toni/miniconda3/envs/research/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()